# Medical & Clinical Guidelines Summarizer

### Built on Week 1, Day 2 — using a local Ollama model (no API costs, data stays on your machine)

**Use case:** Paste any URL from WHO, CDC, NIH, NHS, NICE, or any clinical guidelines page and get a structured plain-language summary — ideal for busy clinicians, medical students, or health researchers.

This extends the Day 1 webpage summarizer by:
- Using a **locally-running open-source LLM** (Llama 3.2 via Ollama) — no data leaves your machine
- Tailoring the prompt specifically for **clinical/medical content**
- Returning a **structured output** with recommendations, who it applies to, and key actions

In [ ]:
import os
import sys
sys.path.append('../..')  # go up to week1/ where scraper.py lives

from dotenv import load_dotenv
from openai import OpenAI
from scraper import fetch_website_contents
from IPython.display import Markdown, display

load_dotenv(override=True)

# Verify Ollama is running
import requests
try:
    r = requests.get("http://localhost:11434")
    print("Ollama is running:", r.content.decode())
except Exception:
    print("Ollama is NOT running. Open a terminal and run: ollama serve")


In [ ]:
# Connect to local Ollama — same OpenAI-compatible interface, zero API cost
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

MODEL = "llama3.2"  # swap to "llama3.2:1b" if your machine is slow

In [ ]:
# System prompt engineered specifically for clinical/medical content

SYSTEM_PROMPT = """
You are a clinical knowledge assistant trained to read medical and public health content.
When given the text of a clinical guideline, health fact sheet, or medical webpage, respond with a structured summary using exactly this format:

## Summary
One or two sentences on what this document covers.

## Who It Applies To
The population or patient group this guidance is relevant for.

## Key Recommendations
Bullet points of the most important clinical recommendations or facts.

## Plain Language Explanation
A 2-3 sentence explanation a patient could understand.

## Action Points for Clinicians
Bullet points of what a healthcare professional should take away from this.

Keep it concise. Do not add disclaimers. Do not wrap output in a code block.
Respond in markdown only.
"""

In [ ]:
def messages_for(content):
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": "Here is the content from a medical/clinical webpage. Please summarize it.\n\n" + content}
    ]

def summarize_guidelines(url):
    print(f"Fetching: {url}")
    content = fetch_website_contents(url)
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages_for(content)
    )
    return response.choices[0].message.content

def display_guidelines_summary(url):
    summary = summarize_guidelines(url)
    display(Markdown(f"---\n### Source: {url}\n---\n{summary}"))

## Test 1 — WHO Diabetes Fact Sheet

In [ ]:
display_guidelines_summary("https://www.who.int/news-room/fact-sheets/detail/diabetes")

## Test 2 — CDC Hypertension Guidelines

In [ ]:
display_guidelines_summary("https://www.cdc.gov/high-blood-pressure/about/index.html")

## Test 3 — NHS Mental Health Guidelines

In [ ]:
display_guidelines_summary("https://www.nhs.uk/mental-health/conditions/depression-in-adults/overview/")

## Try your own URL

Paste any medical/clinical URL below — WHO, CDC, NIH, NHS, NICE, PubMed abstract page, etc.

In [ ]:
my_url = "https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds)"

display_guidelines_summary(my_url)